In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet_f_all'

In [3]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# RUN THE CELLS BELOW HERE IF YOU WANT TO RESHUFFLE

In [4]:
user_numbers = data_f['user_id'].unique()
np.random.shuffle(user_numbers)
dev_user_ids = user_numbers[0:531]
validation_user_ids = user_numbers[531:581]
exploitation_user_ids = user_numbers[581:]

In [5]:
dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
dev_vf = visual_f.loc[dev_df.index]
val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
val_vf = visual_f.loc[val_df.index]
exp_df = data_f.loc[data_f['user_id'].isin(exploitation_user_ids)]
exp_vf = visual_f.loc[exp_df.index]

In [6]:
dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(1,13))]
dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(np.arange(13,25))]

In [7]:
val_df_gen = val_df.loc[val_df['fakeness']==0]
val_df_fake = val_df.loc[val_df['fakeness']==1]
val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(1,13))]
val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(np.arange(13,25))]

In [8]:
DECISION_THRESHOLD = 0.1
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
randoms_sum = 0
randoms_r_count = 0
for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
    clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=True, kernel='linear', random_state=1923, cache_size=200) # gens avg: 0.9433333333333331 fakes_avg: 0.8593333333333333 cumulative_avg: 0.9013333333333332
    
    y_train = y_train=(pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen]))['user_id']==user_id
    X_train = visual_f.loc[y_train.index]  
    clf.fit(X_train, y_train)
    
    y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    # score = accuracy_score(y_valid_f, (clf.predict(X_valid_f)))
    score = accuracy_score(y_valid_f, (clf.predict_proba(X_valid_f)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_f,y_valid_f)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    # score = accuracy_score(y_valid_g, (clf.predict(X_valid_g)))
    score = accuracy_score(y_valid_g, (clf.predict_proba(X_valid_g)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_g,y_valid_g)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    '''
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1
    '''
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
    print ("gens_avg_error:", 100*(1-(float(gens_sum)/gens_r_count)), "fakes_avg_error:", 100*(1-(float(fakes_sum)/fakes_r_count)), 
            "cumulative_avg_error:", 100*(1-((float(gens_sum)/gens_r_count)/2 + (float(fakes_sum)/fakes_r_count)/2)))

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens_avg_error: 0.0 fakes_avg_error: 3.3333333333333326 cumulative_avg_error: 1.6666666666666607


  2%|#6                                                                                 | 1/50 [00:10<08:55, 10.93s/it]

gens_avg_error: 0.0 fakes_avg_error: 23.33333333333334 cumulative_avg_error: 11.66666666666667


  4%|###3                                                                               | 2/50 [00:21<08:39, 10.83s/it]

...

gens_avg_error: 2.298850574712641 fakes_avg_error: 14.367816091954055 cumulative_avg_error: 8.333333333333348


 58%|###############################################5                                  | 29/50 [05:12<03:51, 11.04s/it]

gens_avg_error: 2.2222222222222143 fakes_avg_error: 14.333333333333364 cumulative_avg_error: 8.277777777777784


 60%|#################################################1                                | 30/50 [05:21<03:28, 10.45s/it]

KeyboardInterrupt: 